In [1]:
import polars as pl
import os

os.getcwd()

'/home/user/Python/Baseoffset-Fitting-Manager/notebooks'

In [48]:
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from btc_options.analytics.maths import convert_paramter_into_rate, convert_rate_into_parameter
from scipy.optimize import minimize
import numpy as np
from datetime import datetime

ImportError: cannot import name 'convert_rate_into_parameter' from 'btc_options.analytics.maths' (/home/user/Python/Baseoffset-Fitting-Manager/btc_options/analytics/maths.py)

In [10]:
df = pl.read_csv('synthetic.csv')

In [11]:
my_dt = datetime(2025,8,29,12,32)

In [12]:
df

strike,bid,ask,mid,spread,S,tau,bid_price_fut,ask_price_fut,bid_size,ask_size,bid_size_P,ask_size_P,bid_size_fut,ask_size_fut
i64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,str,str
108000,-3436.76,-3069.73,-3253.25,367.03,111222.07,0.002222,null,null,28.4,28.4,1.0,2.2,null,null
109000,-2424.64,-2046.49,-2235.56,378.16,111222.07,0.002222,null,null,30.5,30.4,13.0,4.2,null,null
110000,-1434.76,-1045.49,-1240.13,389.28,111222.07,0.002222,null,null,49.6,49.7,1.0,0.3,null,null
111000,-333.67,-55.61,-194.64,278.06,111222.07,0.002222,null,null,26.6,29.2,47.8,14.0,null,null
112000,556.11,956.51,756.31,400.4,111222.07,0.002222,null,null,8.5,10.0,58.0,56.6,null,null
113000,1557.11,1957.51,1757.31,400.4,111222.07,0.002222,null,null,5.0,35.9,33.8,33.6,null,null


In [13]:
previous_param = (0.3421, 0.1)  # r and q
tau = df['tau'][0]
S = df['S'][0]

initial_guess_coef = float(np.exp(-previous_param[0] * tau))
initial_guess_const = float(-S * np.exp(-previous_param[1] * tau))

initial_guess = np.array([initial_guess_const, initial_guess_coef])

print(initial_guess)

[-1.11197357e+05  9.99240067e-01]


In [39]:
# y = df["mid"].to_numpy()
# X = df["strike"].to_numpy()
# weight = 1 / df["spread"].to_numpy()
# weight = weight * weight
# lambda_reg = 1

def objective(params, x, y, weight, past_params, lambda_reg, spot, tau):
    const, coef = params
    optimized_rate = convert_paramter_into_rate(params, spot, tau)
    residuals = y - (const + coef * x)
    sse =  np.sum(weight * residuals**2)
    penalty = lambda_reg * np.sum((optimized_rate - past_params)**2)
    print(f"constant={const:10.2f} coefficient={coef:8.4f} r={optimized_rate[0]:10.4f} q={optimized_rate[1]:.4f} sse={sse:18.4f} penalty={penalty:12.4f}")
    return sse + penalty

In [43]:

spot, tau = df['S'][0], df['tau'][0]
initial_rate = convert_paramter_into_rate(initial_guess, spot, tau)
print(f"initial_r={initial_rate[0]:8.4f}  initital_q= {initial_rate[1]:6.4f}")
result =\
minimize(fun=objective, 
         x0=initial_guess, 
         args=(
                (X:=df['strike'].to_numpy()),
                (Y:=df['mid'].to_numpy()),
                (weight:= (1 / df['spread'].to_numpy())**2),
                initial_rate,
                (lambda_reg:=10),
                spot,
                tau
               ),
         method='SLSQP')

initial_r=  0.3421  initital_q= 0.1000
constant=-111197.36 coefficient=  0.9992 r=    0.3421 q=0.1000 sse=            0.1477 penalty=      0.0000
constant=-111197.36 coefficient=  0.9992 r=    0.3421 q=0.1000 sse=            0.1477 penalty=      0.0000
constant=-111197.36 coefficient=  0.9992 r=    0.3421 q=0.1000 sse=            0.1477 penalty=      0.0000
constant=-111197.35 coefficient=531.5628 r=-2824.1196 q=0.1000 sse= 159541419491.8255 penalty=79775837.3688
constant=-111197.36 coefficient= 54.0556 r=-1795.5058 q=0.1000 sse=   1595388858.4298 penalty=32250698.5810
constant=-111197.36 coefficient=  6.3049 r= -828.5954 q=0.1000 sse=     15951355.0670 penalty=6871374.3437
constant=-111197.36 coefficient=  1.5298 r= -191.3127 q=0.1000 sse=       159260.3305 penalty= 367315.6562
constant=-111197.36 coefficient=  1.0523 r=  -22.9387 q=0.1000 sse=         1567.4129 penalty=   5419.9465
constant=-111197.36 coefficient=  1.0045 r=   -2.0409 q=0.1000 sse=           13.2867 penalty=     56.7

In [46]:
convert_paramter_into_rate(result.x, spot, tau)

array([0.30438643, 0.13786739])

In [47]:
result.params

AttributeError: params